# Challenge description

With the LAMBCHOP doomsday device finished, Commander Lambda is preparing to debut on the galactic stage -- but in order to make a grand entrance, Lambda needs a grand staircase! As the Commander's personal assistant, you've been tasked with figuring out how to build the best staircase EVER. 

Lambda has given you an overview of the types of bricks available, plus a budget. You can buy different amounts of the different types of bricks (for example, 3 little pink bricks, or 5 blue lace bricks). Commander Lambda wants to know how many different types of staircases can be built with each amount of bricks, so they can pick the one with the most options. 

Each type of staircase should consist of 2 or more steps.  No two steps are allowed to be at the same height - each step must be lower than the previous one. All steps must contain at least one brick. A step's height is classified as the total amount of bricks that make up that step.
For example, when N = 3, you have only 1 choice of how to build the staircase, with the first step having a height of 2 and the second step having a height of 1: (# indicates a brick)

\#  
\#\#  
21

When N = 4, you still only have 1 staircase choice:

\#  
\#\#\#  
31
 
But when N = 5, there are two ways you can build a staircase from the given bricks. The two staircases can have heights (4, 1) or (3, 2), as shown below:

\#  
\#\#\#\#  
41

\#\#  
\#\#\#  
32

Write a function called solution(n) that takes a positive integer n and returns the number of different staircases that can be built from exactly n bricks. n will always be at least 3 (so you can have a staircase at all), but no more than 200, because Commander Lambda's not made of money!

# Code

## Solution

### Recursive solution with memorization:

In [16]:
def solution_recursive(n):
    def get_staircases(remained_bricks_count, previous_height, memo):
        if (remained_bricks_count, previous_height) in memo:
            return memo[(remained_bricks_count, previous_height)]
        if remained_bricks_count == 0: return 1
        if remained_bricks_count < 0: return 0
        
        staircase_count = 0
        for stair_height in range(1, previous_height):
            count = get_staircases(
                remained_bricks_count - stair_height,
                stair_height,
                memo
                )
            staircase_count += count
        
        memo[(remained_bricks_count, previous_height)] = staircase_count
        return staircase_count
            
    return get_staircases(n, n, {})

In [17]:
assert solution_recursive(3) == 1

In [18]:
assert solution_recursive(4) == 1

In [19]:
assert solution_recursive(5) == 2

In [20]:
assert solution_recursive(200) == 487067745

### Another recursive solution:

Either builds a new stair from current height and then increments the height;
or tries the current stair with incremented height

In [21]:
def solution_recursive_2(n):
    def get_staircases(current_height, remaining_bricks, memo):
        if memo[current_height][remaining_bricks] != -1:
            return memo[current_height][remaining_bricks]

        if remaining_bricks == 0:
            return 1
        if remaining_bricks < current_height:
            return 0
        
        staircase_count = get_staircases(current_height + 1, remaining_bricks - current_height, memo) + \
            get_staircases(current_height + 1, remaining_bricks, memo)
        memo[current_height][remaining_bricks] = staircase_count
        return staircase_count
    
    memo = [[-1 for j in range(n + 2)] for i in range(n + 2)]
    return get_staircases(1, n, memo) - 1

In [22]:
assert solution_recursive_2(3) == 1

In [23]:
assert solution_recursive_2(4) == 1

In [24]:
assert solution_recursive_2(5) == 2

In [25]:
assert solution_recursive_2(200) == 487067745

### Mehdi's DP solution:

In [32]:
map = {}


def build_map(max):
    for s in range(1, max):
        for x in range(1, s):
            bricks = x
            height = s - x
            map[(bricks, height)] = get_staircases(bricks, height)


def get_staircases(bricks, height):
    if bricks == 0:
        return 1
    if bricks < 0:
        return 0
    sum = 0
    for x in range(1, height):
        if (bricks - x == 0):
            count = 1
        elif bricks < x:
            count = 0
        else:
            count = map[(bricks - x, x)]
        sum += count
    return sum


def solution_dp_1(bricks):
    build_map(bricks)
    return get_staircases(bricks, bricks)




In [27]:
assert solution_dp_1(3) == 1

In [28]:
assert solution_dp_1(4) == 1

In [29]:
assert solution_dp_1(5) == 2

In [33]:
print(solution_dp_1(200))
# assert solution_dp_1(200) == 487067745

KeyError: (199, 1)

### DP solution:

In [ ]:

def solution_dp_2(n):
    m = [[0 for i in range(n + 1)] for j in range(n + 1)]
    m[0][0] = 1

    for last in range(1, n + 1):
        for left in range(0, n + 1):
            m[last][left] = m[last - 1][left]
            if left >= last:
                m[last][left] += m[last - 1][left - last]

    return m[n][n] - 1

Tests:

In [ ]:
assert solution_dp_2(3) == 1

In [ ]:
assert solution_dp_2(4) == 1

In [ ]:
assert solution_dp_2(5) == 2

In [ ]:
assert solution_dp_2(200) == 487067745

**Description 1:**
***
Each cell is storing the number of different staircases that can be built at the current state.

As explained in the Top-Down section, a state can be defined as:

 (height of the current stair, number of bricks left)
. For the Bottom-Up approach, a state is one entry in the table and corresponds to one sub-problem.

So what the line

 m[last][left] = m[last - 1][left] 
means is: "At the current state, we initialize the number of different staircases that can be built to be what we had at the previous height (last - 1), with the same number of bricks (left variable).

**Description 2:**
***
A. There's a reason for decreasing 1 option from the final result (for example count(1, n) - 1). It is the scenario of 1 stair which utilizes all the bricks, which's a valid sub-problem according to your Top Down state-definition. It's being removed in retrospect, due to the following constraint: "Every staircase consists of at least two steps".

B. You provided a state-definition to the Top Down approach, however the Bottom Up approach lacks such definition. They aren't equal by any means, the bottom up state-definition is more confusing:
m[last][left] represents the amount of options to build a staircase, s.t. its last step is equal or smaller than 'last', utilizing exactly 'left' bricks. The smaller or equal attribute means, that states aren't disjoint, i.e. there are intersections.

C. Still regarding the Bottom Up approach. It's better be clarified (i.e. logical proof), especially since the state-definition is accumulative:
* Addition I: m[last][left] = m[last - 1][left]:
Each sub-problem having last (rightmost) step height of 'last - 1' at most, which utilizes exactly 'left' bricks, should be counted. In reference to B, because the state definition is accumulative.
* Addition II: m[last][left] += m[last - 1][left - last]:
Each sub-problem with max height of 'last - 1' (possibly less), which utilizes exactly 'left - last' bricks, can be expanded by adding 1 more stair of height 'last' (it will be the rightmost of course).

D. Still regarding the Bottom Up approach:
"The running time ... is a little bit slower than for the custom memoization implementation. In the bottom-up approach, all the states are visited (2 for-loops => 500 ** 2 = 250000 states). In the top-down approach, only the states that are needed are computed"
Well, this is actually up to you, and the confusion comes from lack of well-defined state. Notice that your nested loop visits irrelevant states, in which last > left (impossible to build even 1 step of height last, if you have less than last bricks). In fact, for any last > left, we know that:
m[last][left] = m[left][left]
i.e. increasing last any higher than left, cannot add more (accumulated) options. Wrapping up all that, the bottom-up approach will be faster than top-down (due to no function stack calls), if we replace the nested loop
for left in range(0, n + 1):
with
for left in range(0, last + 1):

E. Especially since recurrence formulas can be tricky / confusing, I'd choose more explicit variable naming:
* 'last_step_height' instead 'last'.
* 'bricks_count' or 'utilized_bricks' instead 'left'.